In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional, Union
from functools import reduce 
from pyntcloud import PyntCloud
from tqdm import tqdm
from matplotlib import rc

plt.rcParams["figure.figsize"] = (8, 6)
# rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 18})
# rc('text', usetex=True)
# rc('text.latex',preamble='\\usepackage[utf8]{inputenc}')
# rc('text.latex',preamble='\\usepackage[russian]{babel}')

In [ ]:
COLOR_BLUE = '#2CBDFE'
COLOR_GREEN = '#47DBCD'
COLOR_PINK = '#F3A0F2'
COLOR_PURPLE = '#9D2EC5'
COLOR_VIOLET = '#661D98'
COLOR_AMBER = '#F5B14C'

COLORS = [COLOR_PURPLE, COLOR_AMBER, COLOR_GREEN, COLOR_PINK, COLOR_VIOLET, COLOR_BLUE]
def get_color(idx: int):
    return COLORS[idx % len(COLORS)]

In [ ]:
def show_bar_plots(hists: np.ndarray, 
                   tick_labels: Optional[List[str]] = None,
                   legend_labels: Optional[List[str]] = None,
                   empty: float = 0.0,
                   ticks_rotation: str = 'horizontal',
                   ylabel: Optional[str] = None,
                   xlabel: Optional[str] = None,
                   horizontal=False):
    if len(hists.shape) == 2:
        n, m = hists.shape
        x = np.arange(m)
        f, ax = plt.subplots(figsize=(25, 5))
        for i, y in enumerate(hists):
            width = (1 - empty) / n
            shift = (i + 0.5) * width - 0.5
            if not horizontal:
                ax.bar(x + shift, y, width=width, align='center', 
                       tick_label=tick_labels if i == len(hists) // 2 else None, label=legend_labels[i], color=get_color(i))
            else:
                ax.barh(x + shift, y, height=width, align='center', 
                        tick_label=tick_labels if i == len(hists) // 2 else None, label=legend_labels[i], color=get_color(i))
    else:
        l, n, m = hists.shape
        x = np.arange(m)
        f, ax = plt.subplots(figsize=(40, 16) if horizontal else (20, 4))
        for j in range(l):
            for i, y in enumerate(hists[j]):
                width = (1 - empty) / n
                shift = (i + 0.5) * width - 0.5
                if not horizontal:
                    ax.bar(x + shift, y, width=width, align='center', 
                           tick_label=tick_labels if i == len(hists) // 2 else None, label=legend_labels[i], color=get_color(j * n + i))
                else:
                    ax.barh(x + shift, y, height=width, align='center', 
                            tick_label=tick_labels if i == len(hists) // 2 else None, label=legend_labels[i], color=get_color(j * n + i))
    plt.xticks(rotation=ticks_rotation)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.legend(loc='upper left') 
    ax.set_axisbelow(True)
    ax.yaxis.grid(linestyle = '--', linewidth = 0.5) # horizontal lines 
    # plt.savefig(f'{ylabel}.png', bbox_inches='tight', dpi=200)
    plt.show()
    
def plot_line_graph(df, parameter_x, parameter_y, versions, descriptors=['fpfh', 'rops', 'shot']):
    for v in versions:
        for d in descriptors:
            df_local = df[(df['version'] == v) & (df['descriptor'] == d)]
            plt.plot(df_local[parameter_x].values, df_local[parameter_y].values, label=d + f' ({v})')
            plt.legend()
    plt.show()

In [ ]:
def show_distances(path, thr=1.5, k=0):
    df = pd.read_csv(path)
    dists = np.sort(df.values.flatten())[-k:]
    print('Number of good correspondences:', np.count_nonzero(dists < thr))
    plt.fill_between(np.arange(len(dists)), np.zeros_like(dists), dists, facecolor='blue', alpha=0.5)
    plt.hlines(thr, 0, len(dists) - 1, color='r')
    plt.show()

In [ ]:
def show_uniquenesses(path, a_max=300):
    df = pd.read_csv(path)
    dists = df.values.flatten()
    dists = np.clip(np.sort(dists), a_min=0, a_max=a_max)
    plt.hist(dists, bins='fd')
    plt.show()

In [ ]:
def min_confusion_matrix(hists1, hists2):
    xx, yy = np.meshgrid(np.arange(len(hists1)), np.arange(len(hists2)))
    return np.abs(np.stack((hists1[xx], hists2[yy]), axis=3).min(axis=3)).sum(axis=2)

def l2_confusion_matrix(hists1, hists2):
    xx, yy = np.meshgrid(np.arange(len(hists1)), np.arange(len(hists2)))
    return np.sqrt(((hists1[xx] - hists2[yy]) ** 2).sum(axis=2))
    
def cosine_confusion_matrix(hists1, hists2):
    xx, yy = np.meshgrid(np.arange(len(hists1)), np.arange(len(hists2)))
    return np.sqrt((hists1[xx] * hists2[yy]).sum(axis=2))
    
def show_confusion_matrix(confusion_matrix, labels1=None, labels2=None, vmin=None, vmax=None):
    res = plt.imshow(confusion_matrix, cmap=plt.cm.jet, vmin=vmin, vmax=vmax, interpolation='nearest')
    plt.colorbar(res)
   
    if labels1 is not None: 
        plt.tick_params(labelbottom=False,labeltop=True)
        plt.xticks(range(len(labels1)), labels1, rotation='vertical')
    if labels2 is not None:
        plt.yticks(range(len(labels2)), labels2)
    plt.show()

In [ ]:
def skip_zeros(hist1, hist2):
    mask = ~np.logical_and(hist1 == 0, hist2 == 0)
    return hist1[mask], hist2[mask]

def analyze_descriptors(path_src, path_tgt, ids_src, ids_tgt, descriptor_size, confusion_matrix_fn, descriptions=[], vmin=None, vmax=None):
    hists_src = pd.read_csv(path_src, header=None, usecols=np.arange(descriptor_size).tolist())
    hists_tgt = pd.read_csv(path_tgt, header=None, usecols=np.arange(descriptor_size).tolist())
    for id_src, id_tgt in zip(ids_src, ids_tgt):
        show_bar_plots(np.vstack((hists_src.iloc[id_src].values, hists_tgt.iloc[id_tgt].values)))
   
    show_confusion_matrix(confusion_matrix_fn(hists_src.iloc[ids_src].values, hists_tgt.iloc[ids_tgt].values),
                          descriptions,
                          descriptions,
                          vmin=vmin, vmax=vmax) 
    show_confusion_matrix(confusion_matrix_fn(hists_src.iloc[ids_src].values, hists_src.iloc[ids_src].values),
                          descriptions,
                          descriptions,
                          vmin=vmin, vmax=vmax) 
    show_confusion_matrix(confusion_matrix_fn(hists_tgt.iloc[ids_tgt].values, hists_tgt.iloc[ids_tgt].values),
                          descriptions,
                          descriptions,
                          vmin=vmin, vmax=vmax) 

In [ ]:
# show_uniquenesses('data/debug/KilwaKisiwani_40r_KilwaKisiwani_43r_uniquenesses_src_random_01.csv')
# show_uniquenesses('data/debug/Hokuyo_4_Hokuyo_8_uniquenesses_src_random_01.csv')
# show_uniquenesses('data/debug/bun000_bun045_uniquenesses_src_random_01.csv')

In [ ]:
from pandas.api.types import is_numeric_dtype

def display_colored(df, columns=[], cmap='PuBu', qmin=0.2, qmax=0.8):
    filtered_columns =  list(filter(lambda c: c in df.columns and is_numeric_dtype(df[c]), columns))
    styler = df.style
    for column in filtered_columns:
        styler.background_gradient(subset=[column], cmap=cmap, vmin=df[column].quantile(qmin), vmax=df[column].quantile(qmax))
    display(styler)

In [ ]:
# df = pd.read_csv('data/debug/test_metrics.csv')
# df = df[['testname', 'metric_corr', 'metric_corr_gt', 'metric_icp', 'metric_icp_gt']]
# df['combination'] = df['metric_corr'] * df['metric_icp']
# df['combination_gt'] = df['metric_corr_gt'] * df['metric_icp_gt']
# df['testname'] = df.testname.apply(lambda s: s[5:7] + '-' + s[16:18])
# display(df.style.background_gradient(subset=['metric_corr', 'metric_icp', 'metric_corr_gt', 'metric_icp_gt', 'combination', 'combination_gt']))
# latex_table = df.to_latex(float_format="%.3f", index=False)
# print(latex_table)

In [ ]:
columns = ['version', 'descriptor','testname', 'use_normals', 'rmse','r_err','t_err','pcd_err', 'normal_diff','voxel_size','normal_radius_coef','feature_radius_coef','pcc', 'correct_correspondences', 'correspondences', 'inliers', 'correct_inliers','filter','threshold','n_random', 'corr_uniformity', 'overlap_rmse']

In [ ]:
# df = pd.read_csv('data/debug/test_results.csv')
# df['pcc'] = df['correct_correspondences'] / df['correspondences']
# display_colored(df, columns)

In [ ]:
def analyze_lrfs(df, parameters, descriptors=['shot', 'rops'], lrf_types=['default', 'gt', 'gravity'], ticks_rotation='horizontal'):
    for desc in descriptors:
        df_desc = df[df['descriptor'] == desc]
        testnames = list(sorted(reduce(np.intersect1d, [df_desc[df_desc['lrf'] == lrf]['testname'].values for lrf in lrf_types])))
        df_desc = df_desc[df_desc['testname'].isin(testnames)]
        df_desc = df_desc.sort_values('testname')
        n, m = len(lrf_types), len(testnames)
        if m == 0:
            continue
        for p in parameters:
            hists = np.zeros((n, m))
            tick_labels = []
            for i, lrf in enumerate(lrf_types):
                for j, testname in enumerate(testnames):
                    hists[i, j] = df_desc[(df_desc['testname'] == testname) & (df_desc['lrf'] == lrf)][p].values[0]     
            show_bar_plots(hists, testnames, lrf_types, empty=0.5, ticks_rotation=ticks_rotation, ylabel=p)

In [ ]:
# df = pd.read_csv('data/debug/test_results.csv')
# df['testname'] = df['testname'] + '_' + df['voxel_size'].astype(str)
# df['pcc'] = df['correct_correspondences'] / df['correspondences']
# df = df[(df['descriptor'] != 'fpfh') & (df['version'] == '03')]
# df['lrf'] = df['lrf'].fillna('default')

# parameters = ['pcc']
# for pname in parameters:
#     d = {desc + t: p for desc, t, p in df[df['lrf'] == 'default'][['descriptor', 'testname', pname]].values}
#     df['default'] = (df['descriptor'] + df['testname']).apply(lambda t: d[t])
#     df['relative_' + pname] = df[pname] / df['default']
# analyze_lrfs(df, ['relative_' + pname for pname in parameters] + ['correct_correspondences'], ticks_rotation='vertical')
# df = df[['version', 'testname', 'descriptor', 'lrf', 'pcc', 'correspondences', 'correct_correspondences', 'inliers', 'pcd_err', 't_err', 'r_err']]
# display_colored(df.sort_values(['testname', 'descriptor']), columns)

In [ ]:
# df = pd.read_csv('data/debug/test_results_office.csv')
# df['testname'] = df['testname'] + '_' + df['voxel_size'].astype(str)
# df['pcc'] = df['correct_correspondences'] / df['correspondences']
# df = df[df['descriptor'] != 'fpfh']
# df['lrf'] = df['lrf'].fillna('default')

# parameters = ['pcc']
# for pname in parameters:
#     d = {desc + t: p for desc, t, p in df[df['lrf'] == 'default'][['descriptor', 'testname', pname]].values}
#     df['default'] = (df['descriptor'] + df['testname']).apply(lambda t: d[t])
#     df['relative_' + pname] = df[pname] / df['default']
# analyze_lrfs(df, ['relative_' + pname for pname in parameters] + ['correct_correspondences'], ticks_rotation='vertical')
# df = df[['version', 'testname', 'descriptor', 'lrf', 'pcc', 'correspondences', 'correct_correspondences', 'inliers', 'pcd_err', 't_err', 'r_err']]
# display_colored(df.sort_values(['testname', 'descriptor']), columns)

### FPFH

In [ ]:
# df = pd.read_csv('data/debug/fpfh_results_02.csv')
# df['pcc'] = df['correct_correspondences'] / df['correspondences']
# df_bun = df[df['testname'] == "bun000_bun045"][columns]
# df_hokuyo = df[df['testname'] == 'Hokuyo_4_Hokuyo_8'][columns]

In [ ]:
# display_colored(df_bun.sort_values('pcd_err'), columns)

In [ ]:
# display_colored(df_hokuyo.sort_values('pcd_err'),columns)

### SHOT

In [ ]:
# df = pd.read_csv('data/debug/shot_results_02.csv')
# df['pcc'] = df['correct_correspondences'] / df['correspondences']
# df_bun = df[df['testname'] == "bun000_bun045"][columns]
# df_hokuyo = df[df['testname'] == 'Hokuyo_4_Hokuyo_8'][columns]

In [ ]:
# display_colored(df_bun.sort_values('pcd_err'), columns)

In [ ]:
# display_colored(df_hokuyo.sort_values('pcd_err'), columns)

### RoPS

In [ ]:
# df = pd.read_csv('data/debug/rops_results_02.csv')
# df['pcc'] = df['correct_correspondences'] / df['correspondences']
# df_bun = df[df['testname'] == "bun000_bun045"][columns]
# df_hokuyo = df[df['testname'] == 'Hokuyo_4_Hokuyo_8'][columns]

In [ ]:
# display_colored(df_bun.sort_values('pcd_err'), columns)

In [ ]:
# display_colored(df_hokuyo.sort_values('pcd_err'), columns)

### With normals

In [ ]:
def with_normals(testname):
    df = pd.read_csv('data/debug/test_results.csv')
    df['pcc'] = df['correct_correspondences'] / df['correspondences']
    df = df[df['testname'] == testname][columns]
    df.loc[df['use_normals'] == 1, 'version'] = '02_normals'
    df = df[df['voxel_size'] == 0.05]
    df = df[['version', 'testname', 'descriptor','pcc', 'use_normals', 'normal_diff', 'normal_radius_coef', 'correct_correspondences', 'correspondences', 'correct_inliers', 'inliers']]
    df = df.sort_values(by=['descriptor', 'normal_radius_coef', 'use_normals'])
    display_colored(df, columns)
    plot_line_graph(df, 'normal_radius_coef', 'correct_correspondences', ['02', '02_surface', '02_normals'], descriptors=['shot'])
    plot_line_graph(df, 'normal_radius_coef', 'correct_correspondences', ['02', '02_surface', '02_normals'], descriptors=['fpfh'])

In [ ]:
# with_normals('kig_058_56_overlap_kig_013_12_overlap')

In [ ]:
# with_normals('kig_077_75_kig_076_74')

In [ ]:
def normal_difference(testname, pcd_type):
    plt.rcParams["figure.figsize"] = (8, 6)
    df = pd.read_csv(f'data/debug/{testname}_normal_difference_{pcd_type}.csv')
    df = df.drop_duplicates()
    df_columns = [c for c in df.columns if c not in ['smoothing']]
    show_confusion_matrix(df[df_columns], labels1=df_columns, labels2=df['smoothing'], vmax=0.35)
    plt.rcParams["figure.figsize"] = (8, 6)

In [ ]:
# normal_difference('kig_058_56_overlap_kig_013_12_overlap', 'src')
# normal_difference('kig_058_56_overlap_kig_013_12_overlap', 'tgt')

In [ ]:
# normal_difference('kig_077_75_kig_076_74', 'src')
# normal_difference('kig_077_75_kig_076_74', 'tgt')

In [ ]:
def build_prc(paths: List[str], legends: List[str], thrs: List[float], reverse=False):
    figs, axes = plt.subplots(nrows=1, ncols=4, figsize=(20,5))
    for legend, path in zip(legends, paths):
        df = pd.read_csv(path)
        if reverse:
            df['distance'] = 1.0 - df['distance']
        precision = []
        recall = []
        f1 = []
        for thr in thrs:
            tp = len(df[(df['is_correct']) & (df['distance'] <= thr)])
            fp = len(df[(~df['is_correct']) & (df['distance'] <= thr)])
            fn = len(df[(df['is_correct']) & (df['distance'] > thr)])
            recall.append(tp / (tp + fn))
            if tp == 0:
                precision.append(0.0)
                f1.append(0.0)
            else:
                precision.append(tp / (fp + tp))
                f1.append(2 * precision[-1] * recall[-1] / (precision[-1] + recall[-1]))
            for ax in axes[:2]:
                ax.axvline(x=thr, color='gray', linestyle='-.', alpha=0.1, linewidth=0.5)
        axes[0].plot(thrs, precision, label=legend)
        axes[1].plot(thrs, recall, label=legend)
        axes[2].plot(recall, precision, label=legend)
        axes[3].plot(thrs, f1, label=legend)
    axes[0].set_xlabel('threshold')
    axes[1].set_xlabel('threshold')
    axes[2].set_xlabel('recall')
    axes[3].set_xlabel('threshold')
    axes[0].set_ylabel('precision')
    axes[1].set_ylabel('recall')
    axes[2].set_ylabel('precision')
    axes[3].set_ylabel('f1 score')
    axes[0].legend()
    axes[1].legend()
    axes[2].legend()
    axes[3].legend()
    plt.show()
    

In [ ]:
# paths = ['data/debug/Hokuyo_4_Hokuyo_8_correspondences_debug_600_fpfh_flann_4_10_default_correspondences_ratio_1_05_prc.csv',
#          'data/debug/Hokuyo_4_Hokuyo_8_correspondences_debug_600_shot_flann_4_10_default_correspondences_ratio_1_05_prc.csv',
#          'data/debug/kig_058_56_kig_013_12_correspondences_debug_500_fpfh_bf_4_15_default_correspondences_ratio_1_05_prc.csv',
#          'data/debug/kig_058_56_kig_013_12_correspondences_debug_500_shot_bf_4_15_default_correspondences_ratio_1_05_prc.csv']
# names = ['Hokuyo_fpfh', 'Hokuyo_shot', 'kig_fpfh', 'kig_shot']
# build_prc(paths, names, thrs=[0.8, 0.825, 0.85, 0.875, 0.9, 0.925, 0.95, 0.975, 1.0])

In [ ]:
# paths = ['data/debug/bun000_bun045_correspondences_debug_8_fpfh_flann_4_10_default_correspondences_cluster_1_05_prc.csv']
# names = ['bun_fpfh']
# build_prc(paths, names, thrs=np.arange(12) / 12 * 0.4 + 0.6, reverse=True)

In [ ]:
# paths = ['data/debug/kig_023_22_kig_024_23_correspondences_debug_500_shot_bf_3_15_default_correspondences_ratio_1_06.csv',
#          'data/debug/kig_023_22_kig_024_23_correspondences_debug_500_shot_bf_3_15_default_correspondences_cluster_1_06.csv']
# names = ['24_ratio_shot', '24_cluster_shot']
# build_prc(paths, names, thrs=np.arange(1, 21) / 20 * 1.0 + 0.0)
# paths = ['data/debug/kig_096_94_kig_094_92_correspondences_debug_500_shot_bf_3_15_default_correspondences_ratio_1_06.csv',
#          'data/debug/kig_096_94_kig_094_92_correspondences_debug_500_shot_bf_3_15_default_correspondences_cluster_1_06.csv']
# names = ['96_ratio_shot', '96_cluster_shot']
# build_prc(paths, names, thrs=np.arange(1, 21) / 20 * 1.0 + 0.0)

In [ ]:
# paths = ['data/debug/kig_058_56_kig_013_12_correspondences_debug_500_fpfh_bf_4_15_default_correspondences_cluster_1_05_prc.csv',
#          'data/debug/kig_058_56_kig_013_12_correspondences_debug_500_shot_bf_4_15_default_correspondences_cluster_1_05_prc.csv']
# names = ['kig_fpfh', 'kig_shot']
# build_prc(paths, names, thrs=np.arange(20) / 20 * 0.8 + 0.2, reverse=True)

In [ ]:
# paths = ['data/debug/Hokuyo_4_Hokuyo_8_correspondences_debug_600_fpfh_flann_4_10_default_correspondences_cluster_2_05_prc.csv',
#          'data/debug/Hokuyo_4_Hokuyo_8_correspondences_debug_600_shot_flann_4_10_default_correspondences_cluster_2_05_prc.csv']
# names = ['Hokuyo_fpfh', 'Hokuyo_shot']
# build_prc(paths, names, thrs=np.arange(20) / 20 * 0.8 + 0.2)
# paths = ['data/debug/kig_058_56_kig_013_12_correspondences_debug_500_fpfh_bf_4_15_default_correspondences_cluster_2_05_prc.csv',
#          'data/debug/kig_058_56_kig_013_12_correspondences_debug_500_shot_bf_4_15_default_correspondences_cluster_2_05_prc.csv']
# names = ['kig_fpfh', 'kig_shot']
# build_prc(paths, names, thrs=np.arange(20) / 20 * 0.8 + 0.2)

In [ ]:
# paths = ['data/debug/Hokuyo_4_Hokuyo_8_correspondences_debug_600_fpfh_flann_4_10_default_correspondences_cluster_1_05_prc.csv',
#          'data/debug/Hokuyo_4_Hokuyo_8_correspondences_debug_600_shot_flann_4_10_default_correspondences_cluster_1_05_prc.csv',
#          'data/debug/kig_058_56_kig_013_12_correspondences_debug_500_fpfh_bf_4_15_default_correspondences_cluster_1_05_prc.csv',
#          'data/debug/kig_058_56_kig_013_12_correspondences_debug_500_shot_bf_4_15_default_correspondences_cluster_1_05_prc.csv']
# names = ['Hokuyo_fpfh', 'Hokuyo_shot', 'kig_fpfh', 'kig_shot']
# build_prc(paths, names, thrs=np.arange(20) / 20 * 0.8 + 0.2, reverse=True)

In [ ]:
# paths = ['data/debug/Hokuyo_4_Hokuyo_8_correspondences_debug_600_fpfh_flann_4_10_default_correspondences_cluster_2_05_prc.csv',
#          'data/debug/Hokuyo_4_Hokuyo_8_correspondences_debug_600_shot_flann_4_10_default_correspondences_cluster_2_05_prc.csv',
#          'data/debug/kig_058_56_kig_013_12_correspondences_debug_500_fpfh_bf_4_15_default_correspondences_cluster_2_05_prc.csv',
#          'data/debug/kig_058_56_kig_013_12_correspondences_debug_500_shot_bf_4_15_default_correspondences_cluster_2_05_prc.csv']
# names = ['Hokuyo_fpfh', 'Hokuyo_shot', 'kig_fpfh', 'kig_shot']
# build_prc(paths, names, thrs=np.arange(20) / 20 * 0.8 + 0.2)

### Weights

In [ ]:
# import os
# import matplotlib.colors as mcolors

# PATH = os.path.join('data', 'debug')
# COLORS = list(mcolors.BASE_COLORS.keys())

# def analyze_weights(testnames, parameters):
#     inliers, inliers_gt, curvatures = [], [], []
#     for i, (testname, ps) in enumerate(zip(testnames, parameters)):
#         inliers.append(pd.read_csv(os.path.join(PATH, testname + '_inliers_' + ps + '.csv')).values.flatten())
#         inliers_gt.append(pd.read_csv(os.path.join(PATH, testname + '_inliers_gt_' + ps + '.csv')).values.flatten())
#         curvatures.append(pd.read_csv(os.path.join(PATH, testname + '_curvatures_' + ps + '.csv')))

#         count_nan = np.count_nonzero(curvatures[i]['k1'].isna())
#         print(f'Curvatures contain {count_nan} NaNs')
#         curvatures[i] = curvatures[i].fillna(0)
#         curvatures[i]['kmax'] = np.maximum(curvatures[i]['k1'].values, curvatures[i]['k2'].values)

#     k = len(testnames)
#     fig, axes = plt.subplots(1, k, figsize=(k * 6, 4), squeeze=False)
    
#     for i in range(k):
#         xs = np.arange((len(curvatures[i])))
#         axes[0][i].plot(xs, np.sort(curvatures[i]['kmax'].values), color=COLORS[i % len(COLORS)])
#         axes[0][i].set_title('All curvatures [%s]' % testnames[i])

#     plt.tight_layout()
#     plt.show()

#     fig, axes = plt.subplots(1, k, figsize=(k * 6, 4), squeeze=False)
#     q = 0.8
    
#     exponential = []
#     log_curvedness = []
#     for i in range(k):
#         xs = np.arange((len(curvatures[i])))
#         Lambda = np.log(1.05) * np.quantile(curvatures[i]['kmax'].values, q)
#         exponential.append(np.exp(-Lambda / curvatures[i]['kmax']))
#         log_curvedness.append(np.log(1 + np.sqrt((curvatures[i]['k1'].values ** 2 + curvatures[i]['k2'].values ** 2) / 2.0)))
        
#         count_nonzero_exponential = np.count_nonzero(exponential[i] > 1e-3)
#         count_nonzero_log_curvedness = np.count_nonzero(log_curvedness[i] > 1e-3)

#         print(f'[{testnames[i]}] exponential weights: {count_nonzero_exponential}/{len(exponential[i])}')
#         print(f'[{testnames[i]}] log curvedness weights: {count_nonzero_log_curvedness}/{len(log_curvedness[i])}')
#         axes[0][i].plot(xs, np.sort(exponential[i]), label='exponential', color=COLORS[i % len(COLORS)])
#         axes[0][i].plot(xs, np.sort(log_curvedness[i]), label='log curvedness', color=COLORS[(i + 4) % len(COLORS)])
#         axes[0][i].set_title('All weights [%s]' % testnames[i])
#         axes[0][i].legend()

#     plt.tight_layout()
#     plt.show()

#     fig, (axes0, axes1) = plt.subplots(2, k, figsize=(k * 6, 2 * 4), squeeze=False)

#     for i in range(k):
#         axes0[i].plot(np.arange(len(inliers[i])), np.sort(exponential[i][inliers[i]]), label='exponential', color=COLORS[i % len(COLORS)])
#         axes1[i].plot(np.arange(len(inliers_gt[i])), np.sort(exponential[i][inliers_gt[i]]), label='exponential', color=COLORS[i % len(COLORS)])

#         axes0[i].plot(np.arange(len(inliers[i])), np.sort(log_curvedness[i][inliers[i]]), label='log curvedness', color=COLORS[(i + 4) % len(COLORS)])
#         axes1[i].plot(np.arange(len(inliers_gt[i])), np.sort(log_curvedness[i][inliers_gt[i]]), label='log curvedness', color=COLORS[(i + 4) % len(COLORS)])
        
#         axes0[i].set_title('Weights of incorrect [%s]' % testnames[i])
#         axes1[i].set_title('Weights of correct [%s]' % testnames[i])

#     plt.tight_layout()
#     plt.show()

#     fig, axes = plt.subplots(1, k + 1, figsize=((k + 1) * 6, 4))
    
#     for i in range(k):
#         incorrect_c = len(inliers[i]) / len(curvatures[i])
#         correct_c = len(inliers_gt[i]) / len(curvatures[i])

#         incorrect_exponential = exponential[i][inliers[i]].sum() / exponential[i].sum()
#         correct_exponential = exponential[i][inliers_gt[i]].sum() / exponential[i].sum()

#         incorrect_log_curvedness = log_curvedness[i][inliers[i]].sum() / log_curvedness[i].sum()
#         correct_log_curvedness = log_curvedness[i][inliers_gt[i]].sum() / log_curvedness[i].sum()
        
#         print('[%s, count]          incorrect / correct: %.5f / %.5f' % (testnames[i], incorrect_c, correct_c))
#         print('[%s, exponential]    incorrect / correct: %.5f / %.5f' % (testnames[i], incorrect_exponential, correct_exponential))
#         print('[%s, log_curvedness] incorrect / correct: %.5f / %.5f' % (testnames[i], incorrect_log_curvedness, correct_log_curvedness))
#         print()
        
#         quantile = np.quantile(curvatures[i]['kmax'].values, q)
#         ms = 1.0 + np.arange(40) / 40 * 2.0
#         correct, incorrect = [], []
#         for m in ms:
#             ws = np.exp(-np.log(m) * quantile / curvatures[i]['kmax'])
#             incorrect.append(ws[inliers[i]].sum() / ws.sum())
#             correct.append(ws[inliers_gt[i]].sum() / ws.sum())
#             axes[i + 1].axvline(x=m, color='gray', linestyle='-.', alpha=0.1, linewidth=0.5)
#         axes[0].plot(ms, np.array(correct) / np.array(incorrect), label=testnames[i], color=COLORS[i % len(COLORS)])
#         axes[i + 1].plot(ms, np.array(correct) / np.array(incorrect), color=COLORS[i % len(COLORS)])
#         axes[i + 1].set_title('Correct / incorrect ratio [%s]' % testnames[i])
    
#     axes[0].set_title('Correct / incorrect ratio')
#     axes[0].legend()
#     plt.tight_layout()
#     plt.show()

# testname = [
#     'kig_016_15_kig_014_13',
#     'kig_022_21_kig_023_22',
#     'kig_023_22_kig_024_23',
#     'kig_061_59_kig_044_43',
#     'kig_097_95_kig_095_93'
# ]
# parameters = [
#     '200_shot_bf_4_15_default_closest_point_cluster_1_log_curvedness_06',
#     '500_shot_bf_3_15_default_correspondences_cluster_1_log_curvedness_06',
#     '500_shot_bf_3_15_default_correspondences_cluster_1_log_curvedness_06',
#     '500_shot_bf_3_15_default_correspondences_cluster_1_log_curvedness_06',
#     '500_shot_bf_3_15_default_correspondences_cluster_1_log_curvedness_06'
# ]
# analyze_weights(testname, parameters)

In [ ]:
# df = pd.read_csv('data/debug/weights/test_metrics.csv')
# df = df[df['testname'].apply(lambda s: not s.startswith('bun'))]
# df['weights'] = df['testname'].apply(lambda s: s[s.rfind('1_') + 2: s.rfind('_06')])
# df['testname'] = df['testname'].apply(lambda s: s[: s.find('metric') -1 ])
# df['ratio'] = df['metric_icp'] / df['metric_icp_gt']
# metric_columns = ['metric_corr', 'metric_corr_gt', 'metric_icp', 'metric_icp_gt', 'ratio']
# df = df[['testname', 'weights'] + metric_columns]
# df = df[df['weights'] == 'constant']
# df['metric_prod'] = df['metric_corr'] * df['metric_icp']
# df['metric_prod_gt'] = df['metric_corr_gt'] * df['metric_icp_gt']
# metric_columns = ['metric_corr', 'metric_corr_gt', 'metric_icp', 'metric_icp_gt', 'ratio', 'metric_prod', 'metric_prod_gt']
# df = df.sort_values(['testname', 'weights'])
# display_colored(df, metric_columns)

In [ ]:
def get_label(t: Tuple) -> str:
    if len(t) == 1:
        return t[0]
    return '(' + ', '.join() + ')'
    
    
def analyze_by(df, by: str, methods: List[str], analyzed_parameters: List[Union[str, Tuple]], descriptors=['shot', 'rops'],  ticks_rotation='horizontal', horizontal=False, y_labels=None, x_label=None):
    for desc in descriptors:
        df_desc = df[df['descriptor'] == desc]
        testnames = list(sorted(reduce(np.intersect1d, [df_desc[df_desc[by] == method]['testname'].values for method in methods])))
        df_desc = df_desc[df_desc['testname'].isin(testnames)]
        df_desc = df_desc.sort_values('testname')
        n, m = len(methods), len(testnames)
        if m == 0:
            continue
        for k, ps in enumerate(analyzed_parameters):
            if type(ps) is not tuple:
                ps = (ps,)
            hists = np.zeros((len(ps), n, m))
            for l, p in enumerate(ps):
                for i, method in enumerate(methods):
                    for j, testname in enumerate(testnames):
                        hists[l, i, j] = df_desc[(df_desc['testname'] == testname) & (df_desc[by] == method)][p].values[0]  
            y_label= get_label(ps) if y_labels is None else y_labels[k]
            show_bar_plots(hists, testnames, methods, empty=0.5, ticks_rotation=ticks_rotation, ylabel=y_label, xlabel=x_label, horizontal=horizontal)

In [ ]:
# df = pd.read_csv('data/debug/test_results_thesis.csv')
# df = df[((df['version'] == 7) & (df['matching'] == 'lr')) |
#         ((df['version'] == 8) & (df['matching'] == 'cluster') & (df['metric'] == 'combination') & (df['lrf'] == 'default')) |
#         ((df['version'] == 8) & (df['matching'] == 'cluster') & (df['metric'] == 'combination') & (df['lrf'] == 'gravity')) ]
# df['type'] = 'симметричная проверка'
# df['type'] = df['type'].where(df['matching'] != 'cluster', 'фильтрация')
# df['type'] = df['type'].where(df['lrf'] != 'gravity', 'фильтрация + гравитация')
# df['testname'] = df['testname'].apply(lambda name: name[5:7] + '-' + name[16:18])
# df['pcc'] = df['correct_correspondences'] / df['correspondences']
# analyze_by(df, by='type', methods=['симметричная проверка', 'фильтрация', 'фильтрация + гравитация'], analyzed_parameters=['pcc', 'correspondences', 'correct_correspondences'], y_labels=['Процент верных соответствий', 'Количество соответствий', 'Количество верных соответствий'], x_label="Пары облаков точек", descriptors=['shot'])
# display(df[df['testname'] == '62-46'])

In [ ]:
# df = pd.read_csv('data/test_measurements.csv')
# df['mae'] = df['mae'] / np.pi * 180
# df['sae'] = df['sae'] / np.pi * 180
# df['success_rate'] = 100 * df['success_rate']
# df['mte'] = 1000 * df['mte']
# df['ste'] = 1000 * df['ste']
# df['mrmse'] = 1000 * df['mrmse']
# df['srmse'] = 1000 * df['srmse']
# df['type'] = 'none'
# df['type'] = df['type'].where(df['testname'].apply(lambda s: 'cluster' not in s), 'cluster')
# df['type'] = df['type'].where(df['testname'].apply(lambda s: 'combination' not in s), 'metric')
# df['type'] = df['type'].where(df['testname'].apply(lambda s: 'gravity' not in s), 'gravity')

In [ ]:
# for t in ['none', 'cluster', 'metric']:
#     for dataset in ['office', 'arch', 'trees']:
#         df_p = df[df['testname'].apply(lambda s: s.startswith(dataset))]
#         df_p = df_p[(df_p['type'] == t)]
#         print(dataset, t, len(df_p), end=' ')
#         print('%.0f' % df_p['success_rate'].mean(), end=' & ')
#         df_p = df_p[(df_p['type'] == t) & (df_p['success_rate']!= 0)]
#         print('%.0f $\\pm$ %.0f' % (df_p['mte'].mean(), df_p['ste'].mean()), end=' & ')
#         print('%.2f $\\pm$ %.2f' % (df_p['mae'].mean(), df_p['sae'].mean()), end=' & ')
#         print('%.0f $\\pm$ %.0f' % (df_p['mrmse'].mean(), df_p['srmse'].mean()), end=' \\\\')
#         print()

In [ ]:
# df = pd.read_csv('data/debug/test_results_thesis.csv')
# df['pcc'] = df['correct_correspondences'] / df['correspondences']
# df['testname'] = df['testname'].apply(lambda name: name[5:7] + '-' + name[16:18])
# df['type'] = 'initial'
# df['type'] = df['type'].where(~((df['matching'] == 'cluster') & (df['lrf'] == 'default')), 'cluster')
# df['type'] = df['type'].where(~((df['matching'] == 'cluster') & (df['lrf'] == 'gravity')), 'cluster + gravity')
# df = df[~((df['metric'] == 'combination') & (df['matching'] == 'cluster') & (df['lrf'] == 'default'))]
# df = df[df['ctf'] == 0]
# analyze_by(df, by='type', methods=['initial', 'cluster', 'cluster + gravity'], analyzed_parameters=['pcc', 'correct_correspondences', 'pcd_err'], descriptors=['shot'])
# df = df[['testname', 'descriptor', 'lrf', 'metric', 'matching', 'ctf', 'voxel_size', 'overlap_rmse', 'correspondences', 'correct_correspondences', 'pcc', 'inliers', 'correct_inliers']]
# display_colored(df.sort_values(['testname', 'lrf', 'matching']), columns)

In [ ]:
df = pd.read_csv('data/debug/test_results.csv')
df = df.fillna('default')
df['type'] = ''
df['type'] = df['type'].where(~((df['alignment'] == 'default') & (df['voxel_size'] == 0.04) & (df['matching'] == 'cluster') & (df['metric'] == 'combination') & (df['lrf'] == 'gravity')), 'baseline')
df['type'] = df['type'].where(df['alignment'] != 'gror', 'gror')
df['type'] = df['type'].where(~((df['alignment'] == 'default') & (df['keypoint'] == 'any') & (df['voxel_size'] == 0.1) & (df['matching'] == 'lr') & (df['metric'] == 'correspondences') & (df['lrf'] == 'default')), 'no keypoints')
df['type'] = df['type'].where(~((df['alignment'] == 'default') & (df['keypoint'] == 'iss') & (df['voxel_size'] == 0.1) & (df['matching'] == 'lr') & (df['metric'] == 'correspondences') & (df['lrf'] == 'default')), 'keypoints')
df = df[df['type'] != '']
for column in ['t_err', 'r_err', 'overlap_rmse']:
    df[column] = pd.to_numeric(df[column])
df = df.drop_duplicates(subset=['testname', 'type'])
df['pcc'] = df['correct_correspondences'] / df['correspondences']
df['testname'] = df['testname'].apply(lambda name: name[5:7] + '-' + name[16:18])
# analyze_by(df, by='type', methods=['baseline', 'gror', 'keypoints', 'no keypoints'], analyzed_parameters=['t_err', 'r_err', 'overlap_rmse'], descriptors=['shot'])
df = df[['version', 'descriptor','testname', 'type', 'r_err','t_err','pcc', 'correct_correspondences', 'correspondences', 'inliers', 'correct_inliers','overlap_rmse']]
display_colored(df.sort_values(['testname', 'type']), columns, cmap='YlGnBu', qmin=0.2, qmax=0.7)

In [ ]:
def analyze_features(path_src, path_tgt):
    features_src = pd.read_csv(path_src)
    features_tgt = pd.read_csv(path_tgt)
    features_diff = features_src - features_tgt
    f1_diff = features_diff['f1']
    f2_diff = features_diff['f2']
    f3_diff = features_diff['f3']
    f4_diff = features_diff['f4']

    fs1 = f1_diff.values
    f1_std = np.std(fs1)
    print(f'f1: {np.count_nonzero(fs1 > f1_std) / len(fs1)}')

    fs2 = f2_diff.values
    f2_std = np.std(fs2)
    print(f'f2: {np.count_nonzero(fs2 > f2_std) / len(fs2)}')

    fs3 = f3_diff.values
    f3_std = np.std(fs3)
    print(f'f3: {np.count_nonzero(fs3 > f3_std) / len(fs3)}')

    plt.hist(f1_diff, bins='rice')
    plt.show()

    plt.hist(f2_diff, bins='rice')
    plt.show()

    plt.hist(f3_diff, bins='rice')
    plt.show()

In [ ]:
# 5994 - левый угол
# 245 - верх спинки
# 1565 - середина спинки
# 5545 - нижний угол

# point_ids = pd.read_csv('data/debug/kig_023_22_chair_kig_022_21_chair_ids_02.csv')
# point_ids.set_index('id_src', drop=False, inplace=True)
# point_ids_src = point_ids['id_src'].values
# point_ids_tgt = point_ids['id_tgt'].values
# ids_src = np.array([5994, 245, 1565, 5545])
# ids_tgt = np.array([point_ids[point_ids['id_src'] == id_src]['id_tgt'].values[0] for id_src in ids_src])
# display(point_ids.loc[ids_src])

In [ ]:
# analyze_descriptors("data/debug/kig_023_22_chair_kig_022_21_chair_histograms_src_02_shot.csv",
#                     "data/debug/kig_023_22_chair_kig_022_21_chair_histograms_tgt_02_shot.csv",
#                     ids_src, ids_tgt,
#                     descriptor_size=352,
#                     confusion_matrix_fn=cosine_confusion_matrix,
#                     descriptions=['left upper', 'upper', 'center', 'left bottom'])

In [ ]:
# analyze_descriptors("data/debug/kig_023_22_chair_kig_022_21_chair_histograms_src_02.csv",
#                     "data/debug/kig_023_22_chair_kig_022_21_chair_histograms_tgt_02.csv",
#                     ids_src, ids_tgt,
#                     descriptor_size=33,
#                     confusion_matrix_fn=min_confusion_matrix)

In [ ]:
# df = pd.read_csv('test_results.csv')
# df = df.fillna('')
# df['pcc'] = df['correct_correspondences'] / df['correspondences']
# df['overlapping'] = [df_overlapping.loc[testname[:testname.rfind('kig') - 1] + '.ply'][testname[testname.rfind('kig'):] + '.ply'] for testname in df['testname']]
# df = df[df['overlapping'] > 0.35]
# df = df[df['pcd_err'] > 0.05]
# df = df[['testname', 'r_err', 't_err', 'pcd_err', 'pcc', 'helpful', 'correct_correspondences', 'correspondences', 'correct_inliers', 'overlapping']].sort_values('pcc')
# df = df.sort_values('pcd_err')
# display_colored(df)

In [ ]:
df = pd.read_csv('data/test_levels.csv')
df = df.fillna('')
df['source'] = df['testname'].apply(lambda s: (s if s[:3] == 'kig' else s[:s.find('_')]) + '.ply')
df['target'] = df['testname'].apply(lambda s: (s if s[:3] == 'kig' else s[s.find('_') + 1:]) + '.ply')
df = df[['source', 'target', 'testname', 'helpful', 'level']]
display(df)
df.to_csv('data/test_levels.csv', index=False)

In [ ]:
df = df[df['level'] == 2]
display(df)
print(np.min(df['overlapping']), np.max(df['overlapping']))

In [ ]:
# very difficult = 3, difficult = 2, middle = 1, easy = 0, no = -1

# df = pd.read_csv('test_results.csv')
# df = df.fillna('')
# df['level'] = ''
# df['level'] = df['level'].where(df['helpful'] == '', other=5) 
# df = df[['testname', 'helpful', 'level']]
# df['level'] = df['level'].where(df['testname'] != 'kig_058_56_kig_013_12', other=2) 
# df['level'] = df['level'].where(df['testname'] != 'kig_090_88_kig_088_86', other=2) 
# df['level'] = df['level'].where(df['testname'] != 'kig_061_59_kig_044_43', other=2) 
# df['level'] = df['level'].where(df['testname'] != 'kig_091_89_kig_089_87', other=3)
# df['level'] = df['level'].where(df['testname'] != 'kig_093_91_kig_092_90', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_044_43_kig_011_10', other=1)
# df['level'] = df['level'].where(df['testname'] != 'kig_045_44_kig_040_39', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_089_87_kig_087_85', other=3)
# df['level'] = df['level'].where(df['testname'] != 'kig_096_94_kig_094_92', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_097_95_kig_095_93', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_024_23_kig_023_22', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_051_50_kig_040_39', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_088_86_kig_079_77', other=-1)
# df['level'] = df['level'].where(df['testname'] != 'kig_059_57_kig_048_47', other=3)
# df['level'] = df['level'].where(df['testname'] != 'kig_087_85_kig_079_77', other=-1)
# df['level'] = df['level'].where(df['testname'] != 'kig_040_39_kig_013_12', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_017_16_kig_015_14', other=-1)
# df['level'] = df['level'].where(df['testname'] != 'kig_016_15_kig_014_13', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_062_60_kig_046_45', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_051_50_kig_011_10', other=3)
# df['level'] = df['level'].where(df['testname'] != 'kig_096_94_kig_095_93', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_051_50_kig_024_23', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_095_93_kig_094_92', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_044_43_kig_039_38', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_092_90_kig_090_88', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_090_88_kig_089_87', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_097_95_kig_094_92', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_090_88_kig_089_87', other=1)
# df['level'] = df['level'].where(df['testname'] != 'kig_046_45_kig_012_11', other=2)
# df['level'] = df['level'].where(df['testname'] != 'kig_040_39_kig_011_10', other=1)
# df['level'] = df['level'].where(df['testname'] != 'kig_089_87_kig_088_86', other=1)
# df['level'] = df['level'].where(df['testname'] != 'kig_048_47_kig_045_44', other=1)
# display(df)
# df.to_csv('test_levels.csv', index=False)

In [ ]:
# df = pd.read_csv('test_results_kig.csv')
# df['pcc'] = df['correct_correspondences'] / df['correspondences']
# display(df[['testname', 'pcc', 'correct_correspondences', 'correspondences']].sort_values(by=['pcc']))

In [ ]:
# df = pd.read_csv('test_levels.csv')
# df = df.fillna('')
# display(df)

In [ ]:
# point_ids = pd.read_csv('data/debug/kig_058_56_window_kig_013_12_window_ids_500_fpfh_flann_02.csv')
# point_ids['distance2'] = (point_ids['x_src'] - point_ids['x_tgt']) ** 2 + \
#                          (point_ids['y_src'] - point_ids['y_tgt']) ** 2 + \
#                          (point_ids['z_src'] - point_ids['z_tgt']) ** 2
# point_ids = point_ids.sort_values(by=['id_src', 'distance2'])
# point_ids.set_index('id_src', drop=False, inplace=True)
# point_ids = point_ids[~point_ids.index.duplicated(keep='first')]
# point_ids_src = point_ids['id_src'].values
# point_ids_tgt = point_ids['id_tgt'].values
# # all_ids_src = [70081, 69637, 55565, 97463, 141934, 133020, 402, 129162, 129173, 77361]
# # ids_src = np.array(list(filter(lambda i: i in point_ids.index, all_ids_src)))
# ids_src = [853, 8921, 5980, 6636, 4196]
# ids_tgt = np.array([point_ids[point_ids['id_src'] == id_src]['id_tgt'].values[0] for id_src in ids_src])
# display(point_ids.loc[ids_src])

In [ ]:
# analyze_descriptors("data/debug/kig_058_56_window_kig_013_12_window_histograms_src_500_fpfh_flann_02.csv",
#                     "data/debug/kig_058_56_window_kig_013_12_window_histograms_tgt_500_fpfh_flann_02.csv",
#                     ids_src, ids_tgt,
#                     descriptor_size=33,
#                     confusion_matrix_fn=min_confusion_matrix,
#                     descriptions=['верх', 'середина', 'край', 'подальше', 'далеко'],
#                     vmin=0, vmax=300)

In [ ]:
# analyze_descriptors("data/debug/kig_058_56_window_kig_013_12_window_histograms_src_500_rops_bf_02.csv",
#                     "data/debug/kig_058_56_window_kig_013_12_window_histograms_tgt_500_rops_bf_02.csv",
#                     ids_src, ids_tgt,
#                     descriptor_size=135,
#                     confusion_matrix_fn=min_confusion_matrix,
#                     descriptions=['верх', 'середина', 'край', 'подальше', 'далеко'],
#                     vmin=0, vmax=1)

In [ ]:
# analyze_descriptors("data/debug/kig_058_56_window_kig_013_12_window_histograms_src_500_shot_bf_02.csv",
#                     "data/debug/kig_058_56_window_kig_013_12_window_histograms_tgt_500_shot_bf_02.csv",
#                     ids_src, ids_tgt,
#                     descriptor_size=352,
#                     confusion_matrix_fn=cosine_confusion_matrix,
#                     descriptions=['верх', 'середина', 'край', 'подальше', 'далеко'],
#                     vmin=0, vmax=1)

In [ ]:
plt.rcParams["figure.figsize"] = (20, 20)

df = pd.read_csv('data/kizhi/normals/downsampled_0.05/overlapping.csv', index_col='reading')
show_confusion_matrix(df.values.astype(np.float64), labels1=df.index, labels2=df.index)

In [ ]:
pcd1 = 'kig_045_44.ply'
pcd2 = 'kig_051_50.ply'
df.index[(df.loc[pcd1] * df.loc[pcd2]).argsort()]